In [143]:
import pandas as pd
import glob
import numpy as np
import pathlib as pl

### 1) Combine all the pks datasets

In [144]:
PATH_TO_PROCESSED_DATA = pl.Path('../datasets/processed')

cases_basic_table_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesBasicTable_excel.xlsx')
cases_development_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesDevelopment_excel.xlsx')
suspects_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]Suspects_excel.xlsx')
len(cases_basic_table_files), len(cases_development_files), len(suspects_files)

(10, 10, 10)

 - first combine all the disticnt tables from all the years to 3 tables throughout the years

In [129]:
df = pd.read_excel(cases_basic_table_files[0], skiprows=8, usecols='B:T', header=None,
              names=["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"])

#df.set_index(df.columns[0], inplace=True)
# Transpose -> signal variables are the column names, predictors are the row keys
df = df.transpose() 
df.insert(loc=0, column='INDEX', value=np.arange(df.shape[0]))
df.insert(loc=1, column="offence or offence category", value=df.index)
df.set_index('INDEX', inplace=True)
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(inplace=True, drop=True)
df
# for sure there is a way how to automate the extraction of the names from various rows, but this seems to be easier

,offence or offence category,total offences,offences against life,murder (sect. 211 PC),other types of murder,robbery attended with murder,sexual murder,"manslaughter and killing another at his/her own request (sects. 212, 213, 216 PC)",manslaughter (sect. 212 PC),less serious case of manslaughter (sect. 213 PC),...,offences against provisions designed to protect young persons,computer crime,environmental crime,environmental crimes pursuant to chapter 29 PC,other environment-related crimes pursuant to the PC,environmental offences pursuant to supplemental penal provisions,street crime,damage to property by graffiti in total,"illegal entry/illegal stay under the Aliens Act (key no 725100, 725700)",hightec/computer crime in the narrower sense
0,recorded cases,6082064.0,2962.0,664.0,602.0,44.0,18.0,1515.0,1491.0,7.0,...,1402.0,73907.0,33448.0,13553.0,5005.0,14890.0,1342905.0,95160.0,135743.0,NaN
1,percentage of all cases,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.2,0.5,0.2,0.1,0.2,22.1,1.6,2.2,NaN
2,attempts number,461117.0,1658.0,415.0,385.0,21.0,9.0,1209.0,1199.0,6.0,...,0.0,8638.0,534.0,273.0,124.0,137.0,77948.0,237.0,2293.0,NaN
3,attempts %,7.6,56.0,62.5,64.0,47.7,50.0,79.8,80.4,85.7,...,0.0,11.7,1.6,2.0,2.5,0.9,5.8,0.2,1.7,NaN
4,"< 20,000 inhabitants",1404723.0,912.0,223.0,201.0,14.0,8.0,416.0,405.0,3.0,...,692.0,15064.0,15547.0,6538.0,2501.0,6508.0,266329.0,20390.0,31095.0,NaN
5,"20,000 up to 100,000 inhabitants",1602590.0,932.0,182.0,168.0,10.0,4.0,490.0,489.0,0.0,...,392.0,17367.0,7364.0,2677.0,1222.0,3465.0,350467.0,24380.0,32487.0,NaN
6,"100,000 up to 500,000 inhabitants",1261205.0,575.0,119.0,109.0,7.0,3.0,335.0,329.0,2.0,...,154.0,12580.0,4386.0,2017.0,575.0,1794.0,287668.0,21477.0,23368.0,NaN
7,"500,000 and more inhabitants",1755141.0,529.0,136.0,120.0,13.0,3.0,267.0,262.0,2.0,...,117.0,16844.0,5991.0,2261.0,681.0,3049.0,434587.0,27961.0,48328.0,NaN
8,unknown inhabitants,58405.0,14.0,4.0,4.0,0.0,0.0,7.0,6.0,0.0,...,47.0,12052.0,160.0,60.0,26.0,74.0,3854.0,952.0,465.0,NaN
9,firearm threatened,4567.0,12.0,5.0,3.0,2.0,0.0,7.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,524.0,0.0,0.0,NaN


In [145]:
basic_table = pd.DataFrame()

base = 0
for i, fname in enumerate(cases_basic_table_files):
    print(fname)
    df_i = pd.read_excel(fname, skiprows=8, usecols='B:T', header=None,
              names=["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"])
    year = fname[3:7]
    
    df_i = df_i.transpose() 
    df_i.insert(loc=0, column='INDEX', value=pd.RangeIndex(base, base + len(df_i), 1))
    
    df_i.insert(loc=1, column="offence or offence category", value=df_i.index)
    df_i = df_i.set_index('INDEX')
    
    df_i.columns = df_i.iloc[0]
    df_i.drop(df_i.index[0], inplace=True)
    
    base = base + len(df_i)
    
    df_i['year'] = [2014 + i] * len(df_i)
    
    print(df_i.index)
    
    basic_table = pd.concat([basic_table, df_i], axis=1)


basic_table.to_csv(PATH_TO_PROCESSED_DATA / 'pks_basic_table.csv')

basic_table.shape 

../datasets/raw/police_stats\pks2014CasesBasicTable_excel.xlsx
Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], dtype='int64', name='INDEX')
../datasets/raw/police_stats\pks2015CasesBasicTable_excel.xlsx
Index([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36], dtype='int64', name='INDEX')
../datasets/raw/police_stats\pks2016CasesBasicTable_excel.xlsx
Index([37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54], dtype='int64', name='INDEX')
../datasets/raw/police_stats\pks2017CasesBasicTable_excel.xlsx
Index([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72], dtype='int64', name='INDEX')
../datasets/raw/police_stats\pks2018CasesBasicTable_excel.xlsx
Index([73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90], dtype='int64', name='INDEX')
../datasets/raw/police_stats\pks2019CasesBasicTable_excel.xlsx
Index([91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,
       1

(180, 10915)